In [ ]:
import sys
import os
# Add the src directory to Python path so model.py can find ssn and net modules
sys.path.append(os.path.abspath('../src'))
import numpy as np
from loguru import logger
import torch
from src.model import model
from src.PDAP import retrain

In [2]:
# load the data
path = '../data_result/raw_data/gauss_cos_31x31.npy'# Initialize the weights
data = np.load(path)
logger.info(f"Loaded data with shape: {data.shape}, dtype: {data.dtype}")

2025-09-22 12:19:58.421 | INFO     | __main__:<module>:4 - Loaded data with shape: (961,), dtype: [('x', '<f8', (2,)), ('dv', '<f8', (2,)), ('v', '<f8')]


In [3]:
# Initialize the parameter
power = 2.1
M = 50 # number greedy insertion selected
num_iterations = 10
loss_weights = (1.0, 0.0)
pruning_threshold = 1e-15

gamma = 5.0
alpha = 1e-5
lr_adam = 1e-5
regularization = (gamma, alpha) 
th = 0.0

In [4]:
# Initialize the models
model_1 = model(activation=torch.relu, power=power, regularization=regularization, optimizer='Adam', loss_weights=loss_weights, th = th)
model_2 = model(activation=torch.relu, power=power, regularization=regularization, optimizer='SSN', loss_weights=loss_weights, th=th, train_outerweights=True)

2025-09-22 12:20:00 | INFO     | src.model:_configure_logger:105 - Model initialized
2025-09-22 12:20:00 | INFO     | src.model:_configure_logger:105 - Model initialized


In [5]:
# prepare the data
data_train, data_valid = model_1._prepare_data(data)


2025-09-22 12:20:00 | INFO     | src.model:_prepare_data:150 - Training set: 864 samples, Validation set: 97 samples
2025-09-22 12:20:00 | INFO     | src.model:_prepare_data:153 - Data ranges - x: [-1.00, 1.00], v: [-0.72, 1.00], dv: [-5.86, 5.86]


In [6]:
from src.greedy_insertion import _sample_uniform_sphere_points, insertion
from src.PDAP import prune_small_weights

In [9]:
history = []
alpha = model_1.alpha

# Track best model across all iterations
best_iteration = 0
best_val_loss = float('inf')
W_hidden, b_hidden = _sample_uniform_sphere_points(M)

# Training loop
for i in range(num_iterations):
    logger.info(f"Iteration {i} - Starting...")
    model_1.train(data_train, data_valid, inner_weights=W_hidden, inner_bias=b_hidden, iterations = 1000, display_every = 100)
    state_1 = model_1.net.state_dict()
    W_hidden, b_hidden, W_out = state_1['hidden.weight'].detach().cpu().numpy(), state_1['hidden.bias'].detach().cpu().numpy(), state_1['output.weight'].detach().cpu().numpy()
    model_2.train(data_train, data_valid, inner_weights=W_hidden, inner_bias=b_hidden, outer_weights=W_out, iterations = 1000, display_every = 100)
    
    # Count and prune small weights
    state_2 = model_2.net.state_dict()
    # Use torch-native ops to count small weights, then convert to int for logging
    small_mask = (state_2['output.weight'].abs().flatten() < pruning_threshold)
    small_count = int(small_mask.sum().item())
    logger.info(f"Small weights count: {small_count}, Pruning...")
    # Prune neurons based on the trained outer weights from model_2
    W_hidden, b_hidden, _ = prune_small_weights(
        W_hidden, 
        b_hidden,
        state_2['output.weight'].detach().cpu().numpy(),
        pruning_threshold,
    )
    
    logger.info(f"Recording...")
    record = {'iteration': int(i), 'artifact': model_2.config, 'num_neurons': W_hidden.shape[0]}
    history.append(record)
    if model_2.config['best_val_loss'] < best_val_loss:
        best_val_loss = model_2.config['best_val_loss']
        best_iteration = i
        logger.info(f"New best model found at iteration {i} with validation loss: {best_val_loss:.6f}")
    
    # Insert neurons and train
    W_to_insert, b_to_insert = insertion(data_train, model_1, M, alpha)
    W_hidden = np.concatenate((W_hidden, W_to_insert), axis=0)
    b_hidden = np.concatenate((b_hidden, b_to_insert), axis=0)

2025-09-22 12:32:21 | INFO     | __main__:<module>:11 - Iteration 0 - Starting...
2025-09-22 12:32:21 | INFO     | src.model:train:307 - Starting network training session
2025-09-22 12:32:21 | INFO     | src.model:_create_network:194 - Creating network with 50 neurons
2025-09-22 12:32:21 | INFO     | src.model:_setup_optimizer:244 - Using Adam optimizer with lr=0.01
2025-09-22 12:32:21 | INFO     | src.model:train:322 - Training hyperparameters: iterations=1000, batch_size=1620, display_every=100
2025-09-22 12:32:21 | INFO     | src.model:train:323 - Loss weights: value=1.0, gradient=0.0
2025-09-22 12:32:21 | INFO     | src.model:train:362 - Epoch 0: Train Loss = 1.151971, Val Loss = 2.131949
2025-09-22 12:32:22 | INFO     | src.model:train:362 - Epoch 100: Train Loss = 0.209801, Val Loss = 0.335750
2025-09-22 12:32:22 | INFO     | src.model:train:362 - Epoch 200: Train Loss = 0.154338, Val Loss = 0.885189
2025-09-22 12:32:23 | INFO     | src.model:train:362 - Epoch 300: Train Loss = 0

In [ ]:
# def save(self):
#     """Save training history."""
#     timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
#     filename = f"training_history_{timestamp}.pkl"
#     filepath = os.path.join(self.stats_dir, filename)
    
#     with open(filepath, 'wb') as f:
#         pickle.dump(self.history, f)
        
#     logger.info(f"Saved training history to {filepath}")
#     return filepath